In [1]:
#imports
from __future__ import division, print_function
import sympy as sm
from sympy import trigsimp
import sympy.physics.mechanics as me
import mpmath as mp
import numpy as np

from sympy.physics.vector import init_vprinting, vlatex
init_vprinting(use_latex='mathjax', pretty_print=False)

# Defining symbolic Variables

n = 1
q = me.dynamicsymbols('q:{}'.format(n + 1))  # generalized coordinates
qdot = me.dynamicsymbols('qdot:{}'.format(n + 1))  #generalized speeds
f = me.dynamicsymbols('f')
m = sm.symbols('m:{}'.format(n + 1))
J = sm.symbols('J:{}'.format(n + 1))
l = sm.symbols('l:{}'.format(n))  # lenght of each pendlum
a = sm.symbols('a:{}'.format(n))  #location of Mass-centers
d = sm.symbols('d1:{}'.format(n + 1))  #viscous damping coef.
g, t = sm.symbols('g t')

# intertial reference frame
In_frame = me.ReferenceFrame('In_frame')

# Origninal Point O in Reference Frame :
O = me.Point('O')
O.set_vel(In_frame, 0)

# The Center of Mass Point on cart :
C0 = me.Point('C0')
C0.set_pos(O, q[0] * In_frame.x)
C0.set_vel(In_frame, qdot[0] * In_frame.x)

cart_inertia_dyadic = me.inertia(In_frame, 0, 0, J[0])
cart_central_intertia = (cart_inertia_dyadic, C0)

cart = me.RigidBody('Cart', C0, In_frame, m[0], cart_central_intertia)

kindiffs = [q[0].diff(t) - qdot[0]]  # entforcing qdot=Omega

frames = [In_frame]
mass_centers = [C0]
joint_centers = [C0]
central_intertias = [cart_central_intertia]

forces = [(C0, f * In_frame.x - m[0] * g * In_frame.y)]

torques=[]

rigid_bodies = [cart]


for i in range(n):
    #Creating new reference frame
    Li = In_frame.orientnew('L' + str(i), 'Axis', [q[i + 1], In_frame.z])
    Li.set_ang_vel(In_frame, qdot[i + 1] * In_frame.z)
    frames.append(Li)

    # Creating new mass point centers
    Pi = mass_centers[-1].locatenew('a' + str(i + 1), a[i] * Li.x)
    Pi.v2pt_theory(joint_centers[-1], In_frame, Li)
    mass_centers.append(Pi)

    #Creating new joint Points
    Jointi = joint_centers[-1].locatenew('jont' + str(i + 1), l[i] * Li.x)
    Jointi.v2pt_theory(joint_centers[-1], In_frame, Li)
    joint_centers.append(Jointi)

    #adding forces
    forces.append((Pi, -m[i + 1] * g * In_frame.y))
    
    #adding torqes
    if i==0 :
        torqueVectori= -d[0] * qdot[1] * In_frame.z
        torques.append((Li, torqueVectori))
    else:
        torqueVectori = -d[i] * (qdot[i+1]-qdot[i])*In_frame.z
        torques.append((Li, torqueVectori))
    
    
    #adding cential inertias 
    IDi = me.inertia(frames[i + 1], 0, 0, J[i + 1])
    ICi = (IDi, mass_centers[i + 1])
    central_intertias.append(ICi)

    LBodyi = me.RigidBody('L' + str(i + 1) + '_Body', mass_centers[i + 1],
                          frames[i + 1], m[i + 1], central_intertias[i + 1])
    rigid_bodies.append(LBodyi)

    kindiffs.append(q[i + 1].diff(t) - qdot[i + 1])
    

#generalized force
loads = torques + forces

#Kane's Method --> Equation of motion
Kane = me.KanesMethod(In_frame, q, qdot, kindiffs)
fr, frstar = Kane.kanes_equations(rigid_bodies, loads)

mass_matrix = trigsimp(Kane.mass_matrix_full)
forcing_vector = trigsimp(Kane.forcing_full)

    

In [2]:
loads

[(L0, - d1*qdot1*In_frame.z),
 (C0, f*In_frame.x - g*m0*In_frame.y),
 (a1, - g*m1*In_frame.y)]

In [3]:
forcing_vector

Matrix([
[                        qdot0],
[                        qdot1],
[   a0*m1*qdot1**2*cos(q1) + f],
[-(a0*g*m1*cos(q1) + d1*qdot1)]])

In [4]:
mass_matrix

Matrix([
[1, 0,              0,              0],
[0, 1,              0,              0],
[0, 0,        m0 + m1, -a0*m1*sin(q1)],
[0, 0, -a0*m1*sin(q1),  J1 + a0**2*m1]])

In [21]:
xdot=mass_matrix.T * forcing_vector

In [22]:
xdot

Matrix([
[                                                                                     qdot0],
[                                                                                     qdot1],
[       a0*m1*(a0*g*m1*cos(q1) + d1*qdot1)*sin(q1) + (m0 + m1)*(a0*m1*qdot1**2*cos(q1) + f)],
[-a0*m1*(a0*m1*qdot1**2*cos(q1) + f)*sin(q1) - (J1 + a0**2*m1)*(a0*g*m1*cos(q1) + d1*qdot1)]])

In [7]:
constants = [l, a[0], m[0], m[1], J[0], J[1], d[0], g]
specified=[f]

In [8]:
numerical_constants = np.array([
    0.32,  # l0
    0.2,  #a0
    3.34,  #m0
    0.8512,  #m1
    0,  #J0
    0.01980,  #J1
    0.00715,  #d
    9.81  #g
])

#
numerical_specified = np.array([0])

In [9]:
parameter_dict = dict(zip(constants, numerical_constants))

In [10]:
linearizer = Kane.to_linearizer()

In [11]:
equilibrium_dict = {q[0]: 0, q[1]: np.pi / 2, qdot[0]:0, qdot[1]:0}

op_point=[equilibrium_dict, parameter_dict]

In [12]:
help(linearizer.linearize)

Help on method linearize in module sympy.physics.mechanics.linearize:

linearize(op_point=None, A_and_B=False, simplify=False) method of sympy.physics.mechanics.linearize.Linearizer instance
    Linearize the system about the operating point. Note that
    q_op, u_op, qd_op, ud_op must satisfy the equations of motion.
    These may be either symbolic or numeric.
    
    Parameters
    ----------
    op_point : dict or iterable of dicts, optional
        Dictionary or iterable of dictionaries containing the operating
        point conditions. These will be substituted in to the linearized
        system before the linearization is complete. Leave blank if you
        want a completely symbolic form. Note that any reduction in
        symbols (whether substituted for numbers or expressions with a
        common parameter) will result in faster runtime.
    
    A_and_B : bool, optional
        If A_and_B=False (default), (M, A, B) is returned for forming
        [M]*[q, u]^T = [A]*[q_in

In [13]:
linearizer.r==sm.Matrix(specified)

True

In [14]:
linearizer.r=sm.Matrix(specified)

In [15]:
linearizer.q==q

False

In [16]:
linearizer.q=q

In [17]:
linearizer.u==qdot

False

In [18]:
linearizer.u=qdot

In [19]:
A, B= linearizer.linearize(op_point=op_point,  A_and_B=True)

In [20]:
A

Matrix([
[0,                                                                            0, 1,                    0],
[0,                                                                            0, 0,                    1],
[0, 9.02165662948104e-18*qdot0' + 2.85360764910888e-18*qdot1' + 1.44535471936607, 0, -0.00618799378239859],
[0, 2.22107420497421e-16*qdot0' + 9.02165662948104e-18*qdot1' + 35.5837094678517, 0,   -0.152344452189785]])